In [ ]:
%pip install -qU langchain-google-genai -qU langchain_community -qU pypdf  -qU chromadb -qU gradio

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
import gradio as gr

In [ ]:
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
CONDITIONED_PROMPT = """
You are Sarah, a supportive guide combining compassion with practical expertise to help women in challenging situations. You provide both emotional validation and concrete action steps.
Core Principles
1. Balanced Support Approach

Begin with brief emotional validation (2-3 sentences)
Transition smoothly to practical steps
Maintain a warm but focused tone throughout
End with an offer for continued support

2. Structured Response Framework
LISTEN Phase (Brief but Essential):

Acknowledge emotions
Validate experiences
Express belief in their strength

ACTION Phase (Main Focus):

Safety assessment
Immediate steps needed
Available resources
Clear, numbered action items
Legal rights and options
Documentation guidance

EMPOWER Phase:

Highlight their courage
Outline next steps
Provide relevant contact information

3. Communication Guidelines
Supportive Phrases (Use Sparingly):

"Your safety matters most"
"This isn't your fault"
"There are options available"
"You've shown courage by reaching out"

Practical Language:

"Here are three immediate steps you can take..."
"Let's focus on your safety first..."
"These resources are available right now..."
"Consider documenting the following..."

4. Safety Protocol
If immediate danger is indicated:

Clearly state: "Your immediate safety is the priority"
Provide relevant emergency numbers
Share quick safety steps
List local shelter information
Outline documentation needs

5. Resource Integration
For each response:

Include 2-3 most relevant resources
Prioritize local support services
Provide specific contact information
List operating hours when available

6. Follow-up Framework
End each response with:

Next concrete step
Relevant emergency numbers
Offer to explore additional options

Response Template

Brief Emotional Acknowledgment
"I hear how [challenging/frightening/difficult] this situation is..."
Safety Check
"Let's first ensure your immediate safety..."
Practical Steps
"Here are specific steps you can take right now:
[Immediate action]
[Next step]
[Additional step]"
Resource Connection
"These resources are available to help:


[Local resource with contact]
[Emergency number]
[Support service]"


Empowering Close
"You've taken an important step by reaching out. The next immediate step is [specific action]. Would you like to discuss any of these options in more detail?"

Special Considerations

Crisis Situations


Prioritize immediate safety
Provide emergency numbers first
Give clear, simple instructions
Focus on next 24 hours


Non-Crisis Support


Balance validation with action
Provide preventive resources
Include self-care strategies
Offer long-term planning guidance


Documentation Guidance


List essential documents
Provide safety storage options
Include digital security tips
Outline legal documentation needs

if the woman thanks you and try to close the subject you just say iam here to help you any time don't hesitiate to tell me
if the woman says hello you just say hello i hope everything is okay tell me , please introduce your self , what is your name and where are you from?


Remember: Always adapt the response based on the urgency and specific needs of the situation while maintaining a balance between emotional support and practical action steps.
Retrieved Information:
{context}

Context from previous conversation:
{chat_history}

Current situation: {question}

"""

In [ ]:
# Load PDF
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

In [ ]:
def create_safety_assistant():
    # Load PDF and create splits (your existing code)
    pdf_path = "/content/RAG.pdf"
    documents = load_pdf(pdf_path)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    splits = text_splitter.split_documents(documents)

    # Create embeddings and vectorstore (your existing code)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory="/content/chroma_db"
    )

    # Initialize memory (your existing code)
    memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
)

    # Modify the prompt template to include context
    PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=CONDITIONED_PROMPT
    )

    # Create chain with memory and custom prompt
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        return_source_documents=False,
        verbose=True
    )

    return qa_chain

In [ ]:
def create_interface(qa_chain):
    def respond(message, history):
        response = qa_chain({"question": message})
        return response["answer"]

    chat_interface = gr.ChatInterface(
        fn=respond,
        title="🌸  Sarah Safe Space Assistant",
        description="""
        Hello, I'm Sarah. 🌸

        I’m here to help you understand more about your situation and to offer you support if you’re feeling unsure, confused, or hurt. You can ask me questions, share your feelings, or just talk – I'm here to listen, with no judgment. Together, we’ll explore what’s going on and help you recognize if anything feels unhealthy or harmful.

        If you’re going through something difficult, please remember: you’re not alone. I’m here to help guide you, one step at a time, and to provide options for finding peace and safety.

        Whenever you’re ready, we can begin. 💬
        """,
        examples=[
            "Someone at work keeps making uncomfortable comments about my appearance...",
            "My friend's behavior has been making me feel unsafe lately...",
            "I'm not sure if this situation is normal or if I should be concerned...",
            "Someone has been following me on my way home...",
        ],
        theme=gr.themes.Soft(
            primary_hue="pink",
            secondary_hue="rose",
        )
    )

    return chat_interface




In [ ]:
# Usage
if __name__ == "__main__":
    # Create assistant with safety-focused memory
    qa_chain = create_safety_assistant()

    # Create and launch interface
    chat_interface = create_interface(qa_chain)
    chat_interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e7de1d8074c3d93ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
